### **Behavioral Cloning Project**

The goals / steps of this project are the following:
* Use the simulator to collect data of good driving behavior
* Build, a convolution neural network in Keras that predicts steering angles from images
* Train and validate the model with a training and validation set
* Test that the model successfully drives around track one without leaving the road
* Summarize the results with a written report

[//]: # (Image References)

[raw_data]: ./Analysis/raw_data_hist.png "Raw data histgram"
[cropping]: ./Analysis/cropping.png "Cropping"
[flipping]: ./Analysis/cropping.png "Flipping"
[brightness]: ./Analysis/brightness_changing.png "brightness_changing"
[preprocess]: ./Analysis/after_preprocess_pipe.png "after preprocess"
[after_generator]: ./Analysis/after_generator.png "after_generator"
[nvidia_structure]: ./Analysis/nvidia_structure.png "nvidia_structure"
[loss]: ./Analysis/training_validation_loss.png "training_validation_loss"

## File Explaination 

#### 1. Submission includes all required files and can be used to run the simulator in autonomous mode

My project includes the following files:
* preprocess.py: contains the pipeline of preprocessing of image which includes resizing, changing brightness, cropping anad flipping.
* model.py: contains the script to create and train the model
* drive.py: drives the car in autonomous mode
* model.h5: contains a trained convolution neural network 
* writeup_report.md or writeup_report.pdf summarizing the results

#### 2. Submission includes functional code
Using the Udacity provided simulator and my drive.py file, the car can be driven autonomously around the track by executing 
```sh
python drive.py model.h5
```

## Collecting training data

To train the model to drive a car in the center of the road, I need to drive the car to keep it in the center of the track. To balance the data, I drove the car clockwise for one lap and conter-clockwise for one lap. 

## Preprocessing data

This is the most important thing that I learned in this project. Though building a network model is crucial and many parameters need to be tuned, the input of the network is more important. Without a good training data, no matter how good a network model is, the training result will not be good. This project is a perfect example to explain this. 


#### 1. Add/subtract offset to steering angle for left and right image
Since the simulator will collect three image for each steering angle, I need to make some offset. I choose to add a constant 2.29 to steering angle when left image is chosen and subtract 2.29 when right image is chosen. 

#### 2. Augmentation
Data collected in two laps is actually not enough for training, so augmentation of raw dataset is necessary. I created:
1. Cropping image 40% from top and 12% from bottom

![](./Analysis/cropping.png)

2. Flipping images randomly (50%) 

![](./Analysis/flipping.png)

3. Changing the brightness randomly (with probability greater than 25%)

![](./Analysis/brightness_changing.png)

4. Resizing of image (160x320 -> 64x64).

![](./Analysis/after_preprocess_pipe.png)

#### 3. Using generator to avoid zero bias
When I drive the car in training mode for data collection, in the most of time I do not have to change the steering angle since it just needs to got straight. It means most of collected the steering angles are zero which makes zero as the bias of all dataset. At begining, I just used all dataset to train the model, and the car just drove straight and never turned. So changing the bias of the data not to be zero is one thing to solve. 

The following figure is the hisgram of raw steering angle data

![](./Analysis/raw_data_hist.png)

Implementing generator is a very good choice. Using generator cannot only randomly select data but also can make  preprocessing data with cpu and training model with GPU at the same time. In the generator, it will first randomly select image and steering pair. Then preprocess it. If the preprocessed steering angle smaller than 0.25, I used np.random.uniform() function to generate a number. If this number is larger than a threshold I set, the image-steering angle pair will be kept. Otherwise, select again.I keep all steering angle larger than 0.25. So this will avoid zeros bias. 

The following is the histgram after implementing generator

![](./Analysis/after_generator.png)

## Model Architecture and Training Strategy

I implemented NVIDIA CNN Structure which consists four convolutional layers and three fully connected layers. 

NVIDIA CNN structure:
![nvidia_structure](./Analysis/nvidia_structure.png)

But I add some additional layers. Before the first convolutional layer, I add a Lambda layer to normalize the input data to change to bias of pixel value of each image to zero. For each convolutional layer, I used stride=2 for both vertical and horizontal and the mode is 'same'. A maxpooling layer follows the convolutional layer. Following the maxpooling, a 'relu' activation layer is added to make the model unlinear. After four convolutional parts, there are three dense(fully connected layers) with 'relu'. activation layers

To reduce overfitting, two dropout functions are added after the final two convolutional layers. The model was also trained and validated on different data sets to ensure that the model was not overfitting. The model was tested by running it through the simulator and ensuring that the vehicle could stay on the track.

The following table shows the final model architecture:

| Layer         | Output Shape	        				  | 
|:----------------- |: ----------------------------------------- | 
| Input         | (None,64,64,3)   					    |
| Normalization    | (None,64,64,3)   						|
| Convolution 5x5   | (None,32,32,24)  |
| Max pooling	   | (None,31,31,24)  				 |
| RELU				| (None,31,31,24)							  |
| Convolution 5x5   | (None,16,16,36)  |
| Max pooling	   | (None,15,15,36)  				 |
| RELU				| (None,15,15,36)							  |
| Convolution 5x5   | (None,8,8,48)  |
| Max pooling	   | (None,7,7,48)  				 |
| RELU				| (None,7,7,48)							  |
| Convolution 3x3   | (None,7,7,64)  |
| Max pooling	   | (None,6,6,64)  				 |
| RELU				| (None,6,6,64)							  |
| DROPOUT			| keep_prob = 0.65							  |
| Convolution 3x3   | (None,6,6,64)  |
| Max pooling	   | (None,5,5,64)  				 |
| RELU				| (None,5,5,64)							  |
| DROPOUT			| keep_prob = 0.65							  |
| Flatten        | (None,1600)
| Fully connected	| (None,1164)       					   |
| RELU				| (None,1164)								  |
| Fully connected	| (None,100)        					   |
| RELU				| (None,100)								  |
| Fully connected	| (None,50)        					   |
| RELU				| (None,50)								  |
| Fully connected	| (None,10)        					   |
| RELU				| (None,10)								  |
| Fully connected	| (None,1)        					   |



The model used an adam optimizer,the learning rate was set to 0.0001


## Training Process
For training, I used Adam optimizer and set learning rate to 0.0001. Instead of using model.fit function, model.fit_generator is implemented in this project. As mentioned above, a generator will use less memory space and can make CPU and GPU work at the same time. This will reduce the training time. 

The generator will generate 64 data (batch size = 64) each time when it is called. For each epoch, I want to train more than 20000 data, so the generator will be called 313 times to generate in total 20032 data. There are 10 epochs in total. 

The final loss is show as below:

![](./Analysis/training_validation_loss.png)


## Results

The model is trained to successfully drive the car in autonomous mode in the simulator. The video is in this [link](https://youtu.be/EMU85g3EKGI) for the first person view. This [link](https://youtu.be/cT3uxYmHrc4) is for third person view. 